**En este notebook, nos enfocamos en hacer una visualización y ejemplificación con python de las transformaciones que le hacemos a los datos mediante el Data Fusion en el proceso ETL.** 

# 1. Transformaciones para datos ambientales (SISAIRE)

## 1.1 Groupby de filas por Estacion, año y semestre, y calculo de promedio de concentracion. Además, conversión de concentracion a numero.

In [62]:
import pandas as pd

# Cargar el archivo limpio de SISAIRE
sisaire = pd.read_csv("SISAIRE_Unificado_Transformado.csv")

# -------------------------------
# 1. Conversión de tipo de dato
# -------------------------------
# Eliminar caracteres no numéricos (como comas, espacios o guiones)
sisaire["Concentracion"] = (
    sisaire["Concentracion"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.replace(r"[^0-9.\-]", "", regex=True)
)

# Convertir a número real (float)
sisaire["Concentracion"] = pd.to_numeric(sisaire["Concentracion"], errors="coerce")

# Verificar nulos después de la conversión
nulos = sisaire["Concentracion"].isnull().sum()
print(f"Valores nulos en 'Concentracion' después de convertir: {nulos}")

# Eliminar filas con concentración nula o inválida
sisaire = sisaire.dropna(subset=["Concentracion"]).reset_index(drop=True)

# -------------------------------
# 2. Agrupación y promedio
# -------------------------------
sisaire_agrupado = (
    sisaire
    .groupby(["Estacion", "Contaminante", "Ano", "Semestre"], as_index=False)
    .agg({"Concentracion": "mean"})
)

# Redondear a 3 decimales
sisaire_agrupado["Concentracion"] = sisaire_agrupado["Concentracion"].round(3)

# -------------------------------
# 3. Guardar archivo final
# -------------------------------
sisaire_agrupado.to_csv("SISAIRE_Promediado_2022_2024.csv", index=False)

print("Transformación completada correctamente.")
print(f"Dimensiones finales: {sisaire_agrupado.shape}")
print("\nVista previa del resultado:")
print(sisaire_agrupado.head(10))


Valores nulos en 'Concentracion' después de convertir: 0
Transformación completada correctamente.
Dimensiones finales: (289, 5)

Vista previa del resultado:
  Estacion Contaminante   Ano  Semestre  Concentracion
0   "USME"          NO2  2022         1         29.498
1   "USME"          NO2  2022         2         30.141
2   "USME"          NO2  2023         1         27.374
3   "USME"          NO2  2023         2         23.004
4   "USME"          NO2  2024         1         25.325
5   "USME"          NO2  2024         2         25.966
6   "USME"           O3  2022         1         27.433
7   "USME"           O3  2022         2         25.133
8   "USME"           O3  2023         1         23.754
9   "USME"           O3  2023         2         28.105


# 2. Transformaciones para datos de salud

## 2.1 Eliminar columnas irrelevantes 

In [44]:
import pandas as pd

# Cargar archivo original
morbilidad = pd.read_csv("Morbilidad_Unificada_2022_2024.csv")

# Conservar solo las columnas requeridas
columnas_utiles = [
    "Ano", "Sexo", "Edad_Quiquenio", "Afiliacion",
    "Localidad", "Enfermedad1", "Enfermedad2",
    "NumeroAfectados", "Semestre"
]
morbilidad = morbilidad[columnas_utiles]

print("Columnas conservadas:")
print(morbilidad.columns.tolist())
print(f"Dimensiones actuales: {morbilidad.shape}")


Columnas conservadas:
['Ano', 'Sexo', 'Edad_Quiquenio', 'Afiliacion', 'Localidad', 'Enfermedad1', 'Enfermedad2', 'NumeroAfectados', 'Semestre']
Dimensiones actuales: (4997613, 9)


## 2.2 Filtrar datos por edad y enfermedades relevantes

In [45]:
import re

# Normalizar los textos a minúsculas sin tildes
def normalizar_texto(texto):
    if isinstance(texto, str):
        texto = texto.lower()
        reemplazos = {
            "á": "a",
            "é": "e",
            "í": "i",
            "ó": "o",
            "ú": "u"
        }
        for tilde, sin_tilde in reemplazos.items():
            texto = texto.replace(tilde, sin_tilde)
    return texto

# Aplicar normalización a las columnas de enfermedades
morbilidad["Enfermedad1_norm"] = morbilidad["Enfermedad1"].apply(normalizar_texto)
morbilidad["Enfermedad2_norm"] = morbilidad["Enfermedad2"].apply(normalizar_texto)

# Filtrar por edad y enfermedades de interés con coincidencia parcial
morbilidad_filtrada = morbilidad[
    (morbilidad["Edad_Quiquenio"].str.strip() == "0 a 4") &
    (
        morbilidad["Enfermedad1_norm"].str.contains(r"bronquitis aguda|neumonia|asma", flags=re.IGNORECASE, na=False) |
        morbilidad["Enfermedad2_norm"].str.contains(r"bronquitis aguda|neumonia|asma", flags=re.IGNORECASE, na=False)
    )
].reset_index(drop=True)

# Eliminar columnas de normalización auxiliares
morbilidad_filtrada = morbilidad_filtrada.drop(columns=["Enfermedad1_norm", "Enfermedad2_norm"])

print(f"Dimensiones después del filtrado: {morbilidad_filtrada.shape}")
print("Vista previa:")
print(morbilidad_filtrada.head())


Dimensiones después del filtrado: (10560, 9)
Vista previa:
    Ano    Sexo Edad_Quiquenio    Afiliacion      Localidad  \
0  2022   Mujer          0 a 4  Contributivo  La Candelaria   
1  2022  Hombre          0 a 4    Subsidiado  La Candelaria   
2  2022  Hombre          0 a 4    Particular  La Candelaria   
3  2022  Hombre          0 a 4    Subsidiado  La Candelaria   
4  2022  Hombre          0 a 4  Contributivo  La Candelaria   

                                         Enfermedad1  \
0                       Influenza [Gripe] Y Neumonia   
1  Enfermedades Cronicas De Las Vias Respiratoria...   
2                       Influenza [Gripe] Y Neumonia   
3                       Influenza [Gripe] Y Neumonia   
4                       Influenza [Gripe] Y Neumonia   

                                         Enfermedad2  NumeroAfectados  \
0       Neumonia Viral, No Clasificada En Otra Parte              4.0   
1                                               Asma              2.0   
2  Neu

## 2.3 Numero de afectados enteros

In [36]:
# Convertir NumeroAfectados a tipo entero
morbilidad_filtrada["NumeroAfectados"] = pd.to_numeric(
    morbilidad_filtrada["NumeroAfectados"], errors="coerce"
).fillna(0).astype(int)

# Guardar archivo limpio
morbilidad_filtrada.to_csv("Morbilidad_Unificada_2022_2024_Limpio.csv", index=False)




Archivo limpio guardado como 'Morbilidad_Unificada_2022_2024_Limpio.csv'
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5508 entries, 0 to 5507
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Ano              5508 non-null   int64 
 1   Sexo             5508 non-null   object
 2   Edad_Quiquenio   5508 non-null   object
 3   Afiliacion       5508 non-null   object
 4   Localidad        5508 non-null   object
 5   Enfermedad1      5508 non-null   object
 6   Enfermedad2      5508 non-null   object
 7   NumeroAfectados  5508 non-null   int64 
 8   Semestre         5508 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 387.4+ KB
None


## 2.4 Creacion de columnas binarias Neumonia, Bronquitis Aguda y Asma, y eliminar Enfermedad1 y Enfermedad2

In [46]:
import re

# Función auxiliar para normalizar texto (elimina tildes y pone en minúscula)
def normalizar_texto(texto):
    if isinstance(texto, str):
        texto = texto.lower()
        reemplazos = {
            "á": "a",
            "é": "e",
            "í": "i",
            "ó": "o",
            "ú": "u"
        }
        for tilde, sin_tilde in reemplazos.items():
            texto = texto.replace(tilde, sin_tilde)
    else:
        texto = ""
    return texto

# Normalizar columnas de texto
morbilidad_filtrada["Enfermedad1_norm"] = morbilidad_filtrada["Enfermedad1"].apply(normalizar_texto)
morbilidad_filtrada["Enfermedad2_norm"] = morbilidad_filtrada["Enfermedad2"].apply(normalizar_texto)

# Crear columnas binarias
morbilidad_filtrada["Neumonia"] = morbilidad_filtrada.apply(
    lambda x: "Sí" if (
        "neumonia" in x["Enfermedad1_norm"] or "neumonia" in x["Enfermedad2_norm"]
    ) else "No",
    axis=1
)

morbilidad_filtrada["Bronquitis_Aguda"] = morbilidad_filtrada.apply(
    lambda x: "Sí" if (
        re.search(r"bronquitis\s*aguda", x["Enfermedad1_norm"]) or
        re.search(r"bronquitis\s*aguda", x["Enfermedad2_norm"])
    ) else "No",
    axis=1
)

morbilidad_filtrada["Asma"] = morbilidad_filtrada.apply(
    lambda x: "Sí" if (
        "asma" in x["Enfermedad1_norm"] or "asma" in x["Enfermedad2_norm"]
    ) else "No",
    axis=1
)

# Eliminar columnas auxiliares
morbilidad_filtrada.drop(columns=["Enfermedad1", "Enfermedad2", "Enfermedad1_norm", "Enfermedad2_norm"], inplace=True)

# Vista previa del resultado
print("Columnas actuales:")
print(morbilidad_filtrada.columns.tolist())
print("\nVista previa:")
print(morbilidad_filtrada.head())



Columnas actuales:
['Ano', 'Sexo', 'Edad_Quiquenio', 'Afiliacion', 'Localidad', 'NumeroAfectados', 'Semestre', 'Neumonia', 'Bronquitis_Aguda', 'Asma']

Vista previa:
    Ano    Sexo Edad_Quiquenio    Afiliacion      Localidad  NumeroAfectados  \
0  2022   Mujer          0 a 4  Contributivo  La Candelaria              4.0   
1  2022  Hombre          0 a 4    Subsidiado  La Candelaria              2.0   
2  2022  Hombre          0 a 4    Particular  La Candelaria              2.0   
3  2022  Hombre          0 a 4    Subsidiado  La Candelaria              2.0   
4  2022  Hombre          0 a 4  Contributivo  La Candelaria              2.0   

   Semestre Neumonia Bronquitis_Aguda Asma  
0         1       Sí               No   No  
1         1       No               No   Sí  
2         1       Sí               No   No  
3         1       Sí               No   No  
4         1       Sí               No   No  

Archivo guardado como 'Morbilidad_Unificada_2022_2024_Binaria.csv'


## 2.5 Estandarización de Localidades

In [51]:
# Función auxiliar para normalizar texto (elimina tildes, pasa a mayúsculas)
def normalizar_localidad(nombre):
    if isinstance(nombre, str):
        reemplazos = {
            "á": "a",
            "é": "e",
            "í": "i",
            "ó": "o",
            "ú": "u",
            "Á": "A",
            "É": "E",
            "Í": "I",
            "Ó": "O",
            "Ú": "U"
        }
        for tilde, sin_tilde in reemplazos.items():
            nombre = nombre.replace(tilde, sin_tilde)
        return nombre.strip().upper()
    return nombre

# Aplicar la normalización sobre la columna Localidad
morbilidad_filtrada["Localidad"] = morbilidad_filtrada["Localidad"].apply(normalizar_localidad)

# Vista previa de los nombres de localidad
print("Vista previa de localidades normalizadas:")
print(morbilidad_filtrada["Localidad"].unique()[:20])

# Guardar el archivo actualizado
morbilidad_filtrada.to_csv("Morbilidad_Unificada_2022_2024_Binaria_Normalizada.csv", index=False)
print("\nArchivo guardado como 'Morbilidad_Unificada_2022_2024_Binaria_Normalizada.csv'")


Vista previa de localidades normalizadas:
['LA CANDELARIA' 'USAQUEN' 'ANTONIO NARIÑO' 'BARRIOS UNIDOS' 'CHAPINERO'
 'BOSA' 'SANTA FE' 'SAN CRISTOBAL' 'USME' 'CIUDAD BOLIVAR' 'TUNJUELITO'
 'ENGATIVA' 'FONTIBON' 'KENNEDY' 'FUERA DE BOGOTA' 'SUBA' 'LOS MARTIRES'
 'PUENTE ARANDA' 'RAFAEL URIBE URIBE' 'TEUSAQUILLO']

Archivo guardado como 'Morbilidad_Unificada_2022_2024_Binaria_Normalizada.csv'


# 3. Transformaciones para datos poblacionales

## 3.1 Estandarización de la columna Localidad

In [64]:
import pandas as pd

# Cargar el archivo
poblacion = pd.read_csv("Poblacion_Bogota_2018_2024_Limpio.csv")

# Función para normalizar nombres
def normalizar_localidad(nombre):
    if isinstance(nombre, str):
        reemplazos = {
            "á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u",
            "Á": "A", "É": "E", "Í": "I", "Ó": "O", "Ú": "U"
        }
        for tilde, sin_tilde in reemplazos.items():
            nombre = nombre.replace(tilde, sin_tilde)
        return nombre.strip().upper()
    return nombre

# Aplicar normalización
poblacion["Localidad"] = poblacion["Localidad"].apply(normalizar_localidad)

# Vista previa
print("Vista previa de localidades normalizadas:")
print(poblacion["Localidad"].unique()[:20])


Vista previa de localidades normalizadas:
['USAQUEN' 'CHAPINERO' 'SANTA FE' 'SAN CRISTOBAL' 'USME' 'TUNJUELITO'
 'BOSA' 'KENNEDY' 'FONTIBON' 'ENGATIVA' 'SUBA' 'BARRIOS UNIDOS'
 'TEUSAQUILLO' 'LOS MARTIRES' 'ANTONIO NARIÑO' 'PUENTE ARANDA'
 'LA CANDELARIA' 'RAFAEL URIBE URIBE' 'CIUDAD BOLIVAR' 'SUMAPAZ']


## 3.2 Usar el la poblacion del area total de las localidades

In [66]:
# Filtrar las filas con área total
poblacion_filtrada = poblacion[poblacion["Area"].str.strip().str.upper() == "TOTAL"].reset_index(drop=True)

print(f"Dimensiones después del filtrado: {poblacion_filtrada.shape}")
print("Vista previa después del filtrado:")
print(poblacion_filtrada.head())


Dimensiones después del filtrado: (360, 5)
Vista previa después del filtrado:
       Localidad   Area   Año  Hombres_0a4  Mujeres_0a4
0        USAQUEN  Total  2018        14707        14326
1      CHAPINERO  Total  2018         4100         3844
2       SANTA FE  Total  2018         3307         3113
3  SAN CRISTOBAL  Total  2018        13760        12979
4           USME  Total  2018        14758        14249


In [67]:
# Asegurar que las columnas sean numéricas
poblacion_filtrada["Hombres_0a4"] = pd.to_numeric(poblacion_filtrada["Hombres_0a4"], errors="coerce")
poblacion_filtrada["Mujeres_0a4"] = pd.to_numeric(poblacion_filtrada["Mujeres_0a4"], errors="coerce")

# Crear nueva columna Total
poblacion_filtrada["Total"] = poblacion_filtrada["Hombres_0a4"] + poblacion_filtrada["Mujeres_0a4"]


print(poblacion_filtrada.head())


       Localidad   Area   Año  Hombres_0a4  Mujeres_0a4  Total
0        USAQUEN  Total  2018        14707        14326  29033
1      CHAPINERO  Total  2018         4100         3844   7944
2       SANTA FE  Total  2018         3307         3113   6420
3  SAN CRISTOBAL  Total  2018        13760        12979  26739
4           USME  Total  2018        14758        14249  29007


## 3.3 Filtrar por año de 2022 a 2024

In [69]:
# Asegurar que la columna AÑO sea numérica
poblacion_filtrada["Año"] = pd.to_numeric(poblacion_filtrada["Año"], errors="coerce")

# Filtrar los años de interés
poblacion_final = poblacion_filtrada[poblacion_filtrada["Año"].isin([2022, 2023, 2024])].reset_index(drop=True)

# Guardar el archivo final
poblacion_final.to_csv("Poblacion_Bogota_2022_2024_Final.csv", index=False)

print("Archivo filtrado por años 2022, 2023 y 2024 guardado como 'Poblacion_Bogota_2022_2024_Final.csv'")
print(f"Dimensiones finales: {poblacion_final.shape}")
print("\nVista previa:")
print(poblacion_final.head())


Archivo filtrado por años 2022, 2023 y 2024 guardado como 'Poblacion_Bogota_2022_2024_Final.csv'
Dimensiones finales: (60, 6)

Vista previa:
       Localidad   Area   Año  Hombres_0a4  Mujeres_0a4  Total
0        USAQUEN  Total  2022        15572        14941  30513
1      CHAPINERO  Total  2022         5128         4649   9777
2       SANTA FE  Total  2022         3627         4057   7684
3  SAN CRISTOBAL  Total  2022        15498        15402  30900
4           USME  Total  2022        17571        16822  34393
